In [ ]:
import datetime
import os
import pickle
import sys
from functools import reduce

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    auc,
    average_precision_score,
    confusion_matrix,
    precision_recall_curve,
    roc_auc_score,
    roc_curve,
)

sys.path.append("../..")

from baseline_models.static.utils import run_model
from drift_detector.experiments import *
from drift_detector.explainer import ShiftExplainer
from drift_detector.plotter import (
    brightness,
    colors,
    colorscale,
    errorfill,
    linestyles,
    markers,
    plot_pr,
    plot_roc,
)
from gemini.utils import *
from gemini_utils.constants import *

### Define Parameters ###

In [ ]:
PATH = "/mnt/nfs/project/delirium/drift_exp/JULY-04-2022/"
DATASET = "gemini"
SAMPLES = [10, 20, 50, 100, 200, 500, 1000]
RANDOM_RUNS = 5
SIGN_LEVEL = 0.05
CALC_ACC = True
TIMESTEPS = 6
AGGREGATION_TYPE = "time"
SCALE = True
CONTEXT_TYPE = "lstm"
MODEL_PATH = "/mnt/nfs/project/delirium/drift_exp/JULY-04-2022/random/random_lstm.pt"
REPRESENTATION = "rf"
DR_TECHNIQUES = [
    "NoRed"
]  # ["NoRed","SRP", "PCA", "kPCA", "Isomap","BBSDs_untrained_FFNN","BBSDs_untrained_LSTM", "BBSDs_trained_LSTM"]
MD_TESTS = [
    "Context-Aware MMD"
]  # ["LSDD", "MMD", "LK","Classifier","Context-Aware MMD"]
HOSPITAL = ["SMH", "MSH", "THPC", "THPM", "UHNTG", "UHNTW", "PMH"]

# Query Data

In [ ]:
admin_data, x, y = get_gemini_data(PATH)

In [ ]:
SHIFT = input("Select shift experiment: ")
OUTCOME = input("Select outcome variable: ")

# Define shift types.
if SHIFT == "ko_shift":
    shifts = ["ko_shift_0.1", "ko_shift_0.5", "ko_shift_1.0"]
elif SHIFT == "small_gn_shift":
    shifts = ["small_gn_shift_0.1", "small_gn_shift_0.5", "small_gn_shift_1.0"]
elif SHIFT == "medium_gn_shift":
    shifts = ["medium_gn_shift_0.1", "medium_gn_shift_0.5", "medium_gn_shift_1.0"]
elif SHIFT == "large_gn_shift":
    shifts = ["large_gn_shift_0.1", "large_gn_shift_0.5", "large_gn_shift_1.0"]
elif SHIFT == "mfa_shift":
    shifts = ["mfa_shift_0.25", "mfa_shift_0.5", "mfa_shift_0.75"]
elif SHIFT == "cp_shift":
    shifts = ["cp_shift_0.25", "cp_shift_0.75"]
elif SHIFT == "small_bn_shift":
    shifts = ["small_bn_shift_0.1", "small_bn_shift_0.5", "small_bn_shift_1.0"]
elif SHIFT == "medium_bn_shift":
    shifts = ["medium_bn_shift_0.1", "medium_bn_shift_0.5", "medium_bn_shift_1.0"]
elif SHIFT == "large_bn_shift":
    shifts = ["large_bn_shift_0.1", "large_bn_shift_0.5", "large_bn_shift_1.0"]

## Run Shift Experiments ##

In [ ]:
if os.path.exists(
    os.path.join(PATH, "_".join([SHIFT, "_".join(HOSPITAL), "mean_dr_md_pval.pkl"]))
):
    with open(
        os.path.join(
            PATH, "_".join([SHIFT, "_".join(HOSPITAL), "mean_dr_md_pval.pkl"])
        ),
        "rb",
    ) as f:
        mean_shifts_dr_md_pval = pickle.load(f)
    with open(
        os.path.join(
            PATH, "_".join([SHIFT, "_".join(HOSPITAL), "mean_dr_md_dist.pkl"])
        ),
        "rb",
    ) as f:
        mean_shifts_dr_md_dist = pickle.load(f)
else:
    mean_shifts_dr_md_pval = np.ones(
        (len(shifts), len(DR_TECHNIQUES), len(MD_TESTS), len(SAMPLES))
    ) * (-1)
    mean_shifts_dr_md_dist = np.ones(
        (len(shifts), len(DR_TECHNIQUES), len(MD_TESTS), len(SAMPLES))
    ) * (-1)

if os.path.exists(
    os.path.join(PATH, "_".join([SHIFT, "_".join(HOSPITAL), "std_dr_md_pval.pkl"]))
):
    with open(
        os.path.join(PATH, "_".join([SHIFT, "_".join(HOSPITAL), "std_dr_md_pval.pkl"])),
        "rb",
    ) as f:
        std_shifts_dr_md_pval = pickle.load(f)
    with open(
        os.path.join(PATH, "_".join([SHIFT, "_".join(HOSPITAL), "std_dr_md_dist.pkl"])),
        "rb",
    ) as f:
        std_shifts_dr_md_dist = pickle.load(f)
else:
    std_shifts_dr_md_pval = np.ones(
        (len(shifts), len(DR_TECHNIQUES), len(MD_TESTS), len(SAMPLES))
    ) * (-1)
    std_shifts_dr_md_dist = np.ones(
        (len(shifts), len(DR_TECHNIQUES), len(MD_TESTS), len(SAMPLES))
    ) * (-1)

for si, shift in enumerate(shifts):
    for di, dr_technique in enumerate(DR_TECHNIQUES):
        for mi, md_test in enumerate(MD_TESTS):
            if np.any(mean_shifts_dr_md_pval[si, di, mi, :] == -1):
                print("{} | {} | {} ".format(shift, dr_technique, md_test))
                try:
                    (
                        mean_p_vals,
                        std_p_vals,
                        mean_dist,
                        std_dist,
                    ) = run_synthetic_shift_experiment(
                        shift=shift,
                        admin_data=admin_data,
                        x=x,
                        y=y,
                        outcome=OUTCOME,
                        hospital=HOSPITAL,
                        path=PATH,
                        aggregation_type=AGGREGATION_TYPE,
                        scale=SCALE,
                        dr_technique=dr_technique,
                        model_path=MODEL_PATH,
                        md_test=md_test,
                        context_type=CONTEXT_TYPE,
                        representation=REPRESENTATION,
                        samples=SAMPLES,
                        dataset=DATASET,
                        sign_level=SIGN_LEVEL,
                        timesteps=TIMESTEPS,
                        random_runs=RANDOM_RUNS,
                        calc_acc=CALC_ACC,
                    )
                    mean_shifts_dr_md_pval[si, di, mi, :] = mean_p_vals
                    std_shifts_dr_md_pval[si, di, mi, :] = std_p_vals
                    mean_shifts_dr_md_dist[si, di, mi, :] = mean_dist
                    std_shifts_dr_md_dist[si, di, mi, :] = std_dist

                except ValueError as e:
                    print("Value Error")
                    pass

In [ ]:
if not os.path.exists(
    os.path.join(PATH, "_".join([SHIFT, "_".join(HOSPITAL), "mean_dr_md_pval.pkl"]))
):
    with open(
        os.path.join(
            PATH, "_".join([SHIFT, "_".join(HOSPITAL), "mean_dr_md_pval.pkl"])
        ),
        "wb",
    ) as f:
        pickle.dump(mean_shifts_dr_md_pval, f)
    with open(
        os.path.join(
            PATH, "_".join([SHIFT, "_".join(HOSPITAL), "mean_dr_md_dist.pkl"])
        ),
        "wb",
    ) as f:
        pickle.dump(mean_shifts_dr_md_dist, f)
if not os.path.exists(
    os.path.join(PATH, "_".join([SHIFT, "_".join(HOSPITAL), "_mean_dr_md_pval.pkl"]))
):
    with open(
        os.path.join(PATH, "_".join([SHIFT, "_".join(HOSPITAL), "std_dr_md_pval.pkl"])),
        "wb",
    ) as f:
        pickle.dump(std_shifts_dr_md_pval, f)
    with open(
        os.path.join(PATH, "_".join([SHIFT, "_".join(HOSPITAL), "std_dr_md_dist.pkl"])),
        "wb",
    ) as f:
        pickle.dump(std_shifts_dr_md_dist, f)

In [ ]:
DIM_RED = input("Select Pre-Processing: ")
MD_TEST = input("Select Two-Sample Testing: ")

fig = plt.figure(figsize=(11, 8))
for si, shift in enumerate(shifts):
    for di, dr_technique in enumerate(dr_techniques):
        for mi, md_test in enumerate(md_tests):
            if dr_technique == DIM_RED and md_test == MD_TEST:
                errorfill(
                    np.array(samples),
                    mean_shifts_dr_md_pval[si, di, mi, :],
                    std_shifts_dr_md_pval[si, di, mi, :],
                    fmt=linestyles[mi] + markers[mi],
                    color=colorscale(colors[si], brightness[mi]),
                    label="%s" % "_".join([shift, dr_technique, md_test]),
                )
plt.xlabel("Number of samples from test")
plt.ylabel("$p$-value")
plt.axhline(y=SIGN_LEVEL, color="k")
plt.legend()
plt.show()